In [1]:
import flopy, os
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

In [2]:
# SET FOLDERS
path = os.path.dirname(os.getcwd())
path_model      = (os.path.join(path,"model"))
path_safefig    = (os.path.join(path,"fig"))

# CHECK IF FOLDER EXIST / IF NOT MAKE FOLDER
if not os.path.isdir(path_safefig):
    os.makedirs(path_safefig)

In [3]:
#get nam list
namList = [x[:-4] for x in os.listdir(path_model) if x[-4:]=='.nam']

#get ucn list
ucnList = [x[:-4] for x in os.listdir(path_model) if x[-4:]=='.ucn']

In [4]:
#nam dropdown widget
namDrop = widgets.Dropdown(
    options=namList,
    value=namList[0],
    description='Nam file:',
    disabled=False,
    layout=widgets.Layout(width='20%')
)

#ucn dropdown widget
ucnDrop = widgets.Dropdown(
    options=ucnList,
    value=ucnList[0],
    description='Conc file:',
    disabled=False,
    layout=widgets.Layout(width='20%')
)

#enable to load saved times
timeBot = widgets.Button(
    description='Load times:',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Show available times',
)

#times dropdown widget
timeDrop = widgets.Dropdown(
    options=[0],
    value=0, #ucnList[0],
    description='Times:',
    disabled=False,
    layout=widgets.Layout(width='20%')
)

#plot widget
plotBot = widgets.Button(
    description='Plot conc.',
    disabled=True,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Plot concentrations at specified layer',
)

#save widget
saveBot = widgets.Button(
    description='Save plot',
    disabled=True,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Save concentration plot',
)

#plot function
def showLayConc(namFile,ucnFile,timeDrop):
    with out:
        out.clear_output()
        m = flopy.modflow.mf.Modflow.load('../model/'+namFile+'.nam',version='mfnwt')
        fig, ax = plt.subplots(figsize=(12,8))
        #modelmap = m.modelgrid.plot(ax=ax)
        modelmap = flopy.plot.PlotMapView(m,ax=ax)
        ucnFile = flopy.utils.binaryfile.UcnFile('../model/'+ucnFile+'.ucn')
        concArray = ucnFile.get_data(totim=timeDrop)
        concArray[concArray == -1.e+30] = np.nan
        modelmap.plot_array(concArray)
        plt.show()
        
#save plot
def saveLayConc(namFile,ucnFile,timeDrop):
    with out:
        out.clear_output()
        m = flopy.modflow.mf.Modflow.load('../model/'+namFile+'.nam',version='mfnwt')
        fig, ax = plt.subplots(figsize=(12,8))
        #modelmap = m.modelgrid.plot(ax=ax)
        fig.patch.set_facecolor('white')
        modelmap = flopy.plot.PlotMapView(m,ax=ax)
        ucnFile = flopy.utils.binaryfile.UcnFile('../model/'+ucnFile+'.ucn')
        concArray = ucnFile.get_data(totim=timeDrop)
        concArray[concArray == -1.e+30] = np.nan
        modelmap.plot_array(concArray)
        figname = str(namFile)+'_conc_'+str(timeDrop)+'.png'
        plt.savefig(os.path.join(path_safefig, figname), dpi=300,  transparent=False)
        plt.show()
    
def listTimes(ucnFile):
    ucnFile = flopy.utils.binaryfile.UcnFile('../model/'+ucnFile+'.ucn')
    return ucnFile.get_times()

#on click function times
def timesButtonClicked(b):
    timeDrop.options = listTimes(ucnDrop.value)
    plotBot.disabled = False
    saveBot.disabled = False
    timeBot.disabled = True
    timeBot.button_style = ''

#on click function plot
def plotButtonClicked(b):
    showLayConc(namDrop.value,ucnDrop.value,timeDrop.value)
    #plotBot.disabled = True
    
    #on click function save
def saveButtonClicked(b):
    saveLayConc(namDrop.value,ucnDrop.value,timeDrop.value)
    #plotBot.disabled = True


plotBot.on_click(plotButtonClicked)

timeBot.on_click(timesButtonClicked)

saveBot.on_click(saveButtonClicked)

display(widgets.HBox([namDrop, ucnDrop, timeBot, timeDrop, plotBot, saveBot]))

out = widgets.Output()
display(out)

Output()